In [2]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
from sklearn.calibration import CalibratedClassifierCV
import lightgbm as lgb
from scipy.special import softmax

df = pd.read_csv("../data/cleaned_race_results.csv")
selected_features = ["total_weight", "age", "sex_c", "sex_f", "sex_g", "sex_h", "sex_m", "speed_mps"]



In [3]:
unique_races = df["race_id"].unique()
train_races, test_races = train_test_split(unique_races, test_size=0.2, random_state=42)
train_df = df[df["race_id"].isin(train_races)]
test_df = df[df["race_id"].isin(test_races)]

In [4]:
def summarize_pairwise_features(race_tensor, race_id, horse_ids, labels, finish_positions):
    num_horses, _, num_features = race_tensor.shape
    summary_vectors = []

    for i in range(num_horses):
        comparisons = np.delete(race_tensor[i], i, axis=0)
        stats = np.concatenate([
            comparisons.mean(axis=0),
            comparisons.std(axis=0),
            comparisons.min(axis=0),
            comparisons.max(axis=0)
        ])
        summary_vectors.append(stats)

    df_summary = pd.DataFrame(summary_vectors)
    df_summary["race_id"] = race_id
    df_summary["horse_id"] = horse_ids
    df_summary["target"] = labels
    df_summary["finish_position"] = finish_positions
    return df_summary

In [5]:
def summarize_pairwise_features_per_race(df):
    all_race_dfs = []
    for race_id, race_df in df.groupby("race_id"):
        race_df = race_df.reset_index(drop=True)

        X = race_df[selected_features].to_numpy()
        race_tensor = X[:, np.newaxis, :] - X[np.newaxis, :, :]

        labels = (race_df["finish_position"] == 1).astype(int).tolist()
        horse_ids = race_df["horse_id"].tolist()
        finish_positions = race_df["finish_position"].tolist()

        summarized = summarize_pairwise_features(
            race_tensor, race_id, horse_ids, labels, finish_positions
        )
        all_race_dfs.append(summarized)

    return pd.concat(all_race_dfs, ignore_index=True)


In [6]:
train_final_df = summarize_pairwise_features_per_race(train_df)
test_final_df = summarize_pairwise_features_per_race(test_df)

X_train = train_final_df.drop(columns=["race_id", "horse_id", "target"])
y_train = train_final_df["target"]

X_test = test_final_df.drop(columns=["race_id", "horse_id", "target"])
y_test = test_final_df["target"]

In [7]:
X_model, X_cali, y_model, y_cali = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [8]:
base_model = lgb.LGBMClassifier(objective='binary', is_unbalance=True, random_state=42)
base_model.fit(X_model, y_model)

[LightGBM] [Info] Number of positive: 670, number of negative: 9762
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001168 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2498
[LightGBM] [Info] Number of data points in the train set: 10432, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.064225 -> initscore=-2.678975
[LightGBM] [Info] Start training from score -2.678975
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

LGBMClassifier(is_unbalance=True, objective='binary', random_state=42)

In [9]:
iso_model = CalibratedClassifierCV(base_model, method='isotonic', cv='prefit')
iso_model.fit(X_cali, y_cali)
iso_proba = iso_model.predict_proba(X_test)[:, 1]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


In [10]:
test_final_df = test_final_df.copy()
test_final_df["proba"] = iso_proba

test_final_df["softmax_proba"] = (
    test_final_df.groupby("race_id")["proba"]
    .transform(lambda x: softmax(x.values))
)

test_final_df["pred"] = test_final_df.groupby("race_id")["softmax_proba"].transform(
    lambda x: (x == x.max()).astype(int)
)

In [11]:
top1_accuracy = (test_final_df["pred"] == test_final_df["target"]).mean()
print("Top-1 Accuracy:", top1_accuracy)

print("\\nClassification Report:")
print(classification_report(test_final_df["target"], test_final_df["pred"]))

ap_score = average_precision_score(test_final_df["target"], test_final_df["softmax_proba"])
print("Average Precision Score:", ap_score)

Top-1 Accuracy: 1.0
\nClassification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3060
           1       1.00      1.00      1.00       204

    accuracy                           1.00      3264
   macro avg       1.00      1.00      1.00      3264
weighted avg       1.00      1.00      1.00      3264

Average Precision Score: 1.0
